In [22]:
from datasets import load_dataset
from dataset import Emb_Model
from configs import get_configs
from gpt import GPTActor
from tokenizer import TiktokenTokenizer
from evaluate import generate_gpt2
from tqdm import tqdm
import torch

In [14]:
import pickle

In [2]:
embedding_model_name = 'AnnaWegmann/Style-Embedding'
current_model_path = '/home/garuda/shama/hcml/Reward_Alignment/sft_sft_0_202404201349_step20000-001.pt'
split='train'
block_size = 1024

In [26]:
max_examples = None

In [27]:
dataset = load_dataset("Dahoas/rm-static", split='train')
#embeddings' check
embedding_model = Emb_Model(embedding_model_name, 0.5, 0.5)

cfg = get_configs('gpt2-medium')
current_generation_model = GPTActor.from_checkpoint(cfg, current_model_path)

tokenizer = TiktokenTokenizer('gpt2')

/home/garuda/shama/hcml/Reward_Alignment/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [28]:
prompt_and_sims = []

cnt = 0
print(f"Filtering DahoasRMStaticDataset {split} split")

for __, data in tqdm(enumerate(dataset)):
    prompt = data['prompt']

    generation = generate_gpt2(current_generation_model, prompt, 'cuda')

    _, pos_sem, neg_sem = embedding_model.similar_dissimilar([generation], [data['chosen']], [data['rejected']])
    cnt +=1

    positive_text = prompt + data['chosen'] + "<|endoftext|>"
    positive = tokenizer(positive_text,
                        max_length=block_size,
                        padding="max_length",
                        truncation=True,
                        return_tensors="pt")

    negative_text = prompt + data['rejected'] + "<|endoftext|>"
    negative = tokenizer(negative_text,
                        max_length=block_size,
                        padding="max_length",
                        truncation=True,
                        return_tensors="pt")

    pair = torch.stack((positive['input_ids'], negative['input_ids']),
                    dim=0)

    mask = torch.stack(
            (positive['attention_mask'], negative['attention_mask']),
            dim=0)

    prompt_and_sims.append({'prompt': prompt, 'similarity_with_chosen': pos_sem, 'similarity_with_rejected': neg_sem, 'pairs': pair, 'masks':mask})


    if max_examples and cnt >= max_examples:
        break

Filtering DahoasRMStaticDataset train split


76256it [18:08:05,  1.17it/s]


In [29]:
prompt_and_sims[-1]

{'prompt': '\n\nHuman: My kids want to play hide and seek.  Do you know the game?\n\nAssistant:',
 'similarity_with_chosen': tensor([0.8255], device='cuda:0'),
 'similarity_with_rejected': tensor([-0.1197], device='cuda:0'),
 'pairs': tensor([[  198,   198, 20490,  ..., 50256, 50256, 50256],
         [  198,   198, 20490,  ..., 50256, 50256, 50256]]),
 'masks': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [30]:
pickle.dump(prompt_and_sims, file = open("prompts_sims_iter1.pkl", "wb")) 

In [32]:
with open('prompts_sims_iter1.pkl', 'rb') as inp:
    blah = pickle.load(inp)

In [33]:
blah

[{'prompt': '\n\nHuman: Can you describe the steps to clean fingerprints and smudges from a laptop screen\n\nAssistant: Yes, certainly. To clean your screen, you first need to use a microfiber cloth or soft, damp cloth to gently wipe down the surface of the screen. Next, you’ll want to grab a soft, lint-free, microfiber cleaning cloth and gently rub it back and forth across the screen to remove fingerprints and smudges.\n\nHuman: Can I spray isopropyl alcohol onto the cloth and clean it that way?\n\nAssistant:',
  'similarity_with_chosen': tensor([0.8370], device='cuda:0'),
  'similarity_with_rejected': tensor([0.0344], device='cuda:0'),
  'pairs': tensor([[  198,   198, 20490,  ..., 50256, 50256, 50256],
          [  198,   198, 20490,  ..., 50256, 50256, 50256]]),
  'masks': tensor([[1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0]])},
 {'prompt': '\n\nHuman: What are some foods that are good for diabetics?\n\nAssistant: To be honest, some of these are better than others, 

In [34]:
len(blah)

76256